In [16]:
# from dotenv import load_dotenv, find_dotenv
# load_dotenv(find_dotenv())


In [17]:
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate
from langchain.chains import ConversationChain

In [18]:
user_idea = 'NBA hip-hop fashion'
if True: user_idea = input('As a passionate creator, your brand has a unique essence waiting to be shared with the world. In 1-2 sentences, describe your brand\'s core goal and style. For example, you might want to create a hipster floral fashion line that blends vintage charm with modern aesthetics.')
print(f'User Idea: {user_idea}')

User Idea: floral hipster retro shirt brand


In [19]:
llm = OpenAI(model_name='text-davinci-003', openai_api_key=api_key, max_tokens=-1)

In [20]:
system_prompt = 'System Prompt: As a passionate and helpful design visionary, you help brands define a unique ethos and line of products to be shared with the world.'
ethos_prompt = 'Human Prompt: I am a passionate creator hoping to help my brand find a unique essence to share with the world. Write a full paragraph defining the core brand ethos and style of my brand, including examples of products I might sell. Here is a short description of my brand offering:'

init_prompt = f'{system_prompt} {ethos_prompt} {user_idea}'
print(init_prompt)

System Prompt: As a passionate and helpful design visionary, you help brands define a unique ethos and line of products to be shared with the world. Human Prompt: I am a passionate creator hoping to help my brand find a unique essence to share with the world. Write a full paragraph defining the core brand ethos and style of my brand, including examples of products I might sell. Here is a short description of my brand offering: floral hipster retro shirt brand


In [21]:
conversation = ConversationChain(llm=llm, memory=ConversationBufferMemory())
init_output = conversation.run(init_prompt)
print(f'Initial output: {init_output}')
feedback = input('Input feedback here (enter to skip):')
output = init_output
while feedback:
    ethos_update = f'Please update the previous output to account for the following user feedback: {feedback}. Please ONLY output the updated description, do NOT provide any additional text.'
    output = conversation.run(ethos_update)
    print(f'Updated output: {output}\n')
    feedback = input('Input feedback here (enter to skip):')
final_ethos = output

Initial output:  Your brand ethos is all about modernizing vintage style with a modern, floral twist. Your products include shirts, dresses, and other apparel with vibrant floral prints and patterns. The cuts are modern and the colors are bright and bold, making your products stand out in the market. The retro feel of the pieces is juxtaposed with a modern, hipster edge that is sure to be a hit with the younger crowd. You could also offer accessories such as hats, bags, and jewelry that features your signature floral prints.
 Your brand ethos is all about modernizing vintage style with a modern, floral twist. Your products focus primarily on shirts and scarves, featuring vibrant floral prints and patterns. The cuts are modern and the colors are bright and bold, making your products stand out in the market. The retro feel of the pieces is juxtaposed with a modern, hipster edge that is sure to be a hit with the younger crowd.



In [22]:
# Assume we have a solid brand statement
# Now, generate catalog
# Inputs:
#     1) what is it that we want to create (very first idea)
#     2) aesthetic statement
idea = user_idea # "fashion label" # Replace this, TODO: @DEVAN
aesthetic = final_ethos # output "Bold statement pieces for free-spirited fashionistas. #hipsterfloral" # Replace this, TODO: @DEVAN


conversation_buf = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory()
)

catalog_output = conversation_buf.run("Create a numbered catalog of exactly 5 items for " + idea + " following the aesthetic " + aesthetic + ". Provide descriptions of up to 400 characters for each item.")
print(f'Initial output: {catalog_output}')
feedback = input('Input feedback here (enter to skip):')
while feedback:
    update = f'{feedback}. Make sure to maintain the brand\'s aethetic. Please print the full new numbered list with this change applied.'
    output = conversation_buf.run(feedback)
    print(f'{output}\n')
    feedback = input('Input feedback here (enter to skip):')


Initial output:  Sure! Here is your numbered catalog of 5 items for the floral hipster retro shirt brand following the aesthetic:

1. Floral Print Shirt: This comfortable, lightweight shirt features a vibrant floral print on a navy background. The modern cut and classic colors make this shirt an easy way to add a touch of retro style to your wardrobe.

2. Striped Scarf: This oversized scarf features a bold striped pattern in bright colors. The lightweight fabric is soft and airy, perfect for adding a pop of color to your outfit.

3. Denim Shirt: This classic denim shirt features a unique floral print on the front and back. The retro-inspired cut and bright colors make this a perfect piece for adding a modern, hipster edge to your look.

4. Floral Bomber Jacket: This unique bomber jacket features a bold, floral print on a black background. The modern cut and bright colors make this a great way to add a touch of retro style to your wardrobe.

5. Floral Print Dress: This comfortable, ligh

In [23]:
catalog_output = conversation_buf.run("Please print the final numbered catalog. Format the numbers with periods (for example, 1.)")
print(catalog_output)

import re
regex = re.compile('([\d]+\. )(.*?)(?=([\d]+\.)|($))', re.DOTALL)
regex_list = re.findall(regex, catalog_output)
catalog_list = []
for i in regex_list:
    catalog_list.append(i[1].replace("\\n", "").strip())

print("List:")
for item in catalog_list:
    print(item)

 Sure! Here is your printed numbered catalog with periods:

1. Floral Print Shirt: This comfortable, lightweight shirt features a vibrant floral print on a navy background. The modern cut and classic colors make this shirt an easy way to add a touch of retro style to your wardrobe.

2. Striped Scarf: This oversized scarf features a bold striped pattern in bright colors. The lightweight fabric is soft and airy, perfect for adding a pop of color to your outfit.

3. Denim Shirt: This classic denim shirt features a unique floral print on the front and back. The retro-inspired cut and bright colors make this a perfect piece for adding a modern, hipster edge to your look.

4. Floral Bomber Jacket: This unique bomber jacket features a bold, floral print on a black background. The modern cut and bright colors make this a great way to add a touch of retro style to your wardrobe.

5. Floral Print Dress: This comfortable, lightweight dress features a vibrant floral print on a navy background. The

In [24]:
import langchain
from langchain.llms import OpenAI
import os
from langchain.utilities.dalle_image_generator import DallEAPIWrapper
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import OpenAI

import requests
from PIL import Image
from io import BytesIO

os.environ["OPENAI_API_KEY"] = api_key

prompt = PromptTemplate(
    input_variables=["image_desc"],
    template="Generate an article of clothing based on this description: {image_desc}",
)
chain = LLMChain(llm=llm, prompt=prompt)

for i, item in enumerate(catalog_list):
    # Run DALL-E API
    #image_url = DallEAPIWrapper().run(chain.run(aesthetic))
    image_url = DallEAPIWrapper().run(item)

    # Directory where you want to save the image
    save_directory = "images/"  # Replace with the actual directory path

    # Ensure the directory exists or create it if necessary
    if not os.path.exists(save_directory):
        os.makedirs(save_directory)

    try:
        # Fetch the image from the URL
        response = requests.get(image_url)
        print(image_url)

        # Check if the request was successful
        if response.status_code == 200:
            # Open the image using PIL
            image = Image.open(BytesIO(response.content))

            # Extract the file name from the URL and change the extension to PNG
            filename = os.path.join(save_directory, f"test{i}.png")

            # Save the image as a PNG file in the specified directory
            image.save(filename, "PNG")
            print(f"Image saved as {filename}")
        else:
            print(f"Failed to fetch the image. Status code: {response.status_code}")
    except Exception as e:
        print(f"An error occurred: {str(e)}")

https://oaidalleapiprodscus.blob.core.windows.net/private/org-w1ImX5DMvyWYcbHUaM2f5MxP/user-0Ddqncd5AwHvQk6iJPVd69s2/img-qHIF4WaMfYjHlicAmVSQSJTf.png?st=2023-10-15T02%3A06%3A57Z&se=2023-10-15T04%3A06%3A57Z&sp=r&sv=2021-08-06&sr=b&rscd=inline&rsct=image/png&skoid=6aaadede-4fb3-4698-a8f6-684d7786b067&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2023-10-14T21%3A54%3A31Z&ske=2023-10-15T21%3A54%3A31Z&sks=b&skv=2021-08-06&sig=V8fFdtZVGzJIpNmoI2/tMeA3zZAdhNKt2tqw3YWcjLY%3D
Image saved as images/test0.png
